# Tweets about eating disorders
## Modelling with CV
 

In [1]:
import os
import pandas as pd
import numpy as np
import json
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
from ast import literal_eval
'''
tweets = pd.read_csv('ed-dataset-falcon_spacy2-embeddings-sentence.csv', sep=';', encoding='utf8', converters=
                           {
                            'entities_instances_wikidata':literal_eval,
                            'spacy_entities_ids':literal_eval,
                            'spacy_entities_labels':literal_eval,
                            'falcon_spacy_entities':literal_eval,
                            'falcon_spacy_labels':literal_eval,
                            'falcon_spacy_embeddingsmd4_mw50_RW':literal_eval,
                            'falcon_spacy_embeddingsmd2_mw100_RW':literal_eval,
                            'sent_embedding_1':literal_eval,
                            'sent_embedding_2':literal_eval},error_bad_lines=False)

'''
tweets = pd.read_csv('ed-dataset-falcon_spacy2-embeddings-sentence-md4.csv', sep=';', encoding='utf8', converters=
                           {
                            'falcon_spacy_embeddingsmd4_mw50_RW':literal_eval,
                            'sent_embedding_1':literal_eval},error_bad_lines=False)

In [2]:
import pandas as pd
import numpy as np
import spacy
import nltk
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import regex as re
import string
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel
import io

punctuations = "¡!#$%&'()*+,-./:;<=>¿?@[\]^_`{|}~"
def read_txt(filename):
    list = []
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.readlines()
        for line in data:
            list.append(str(line).replace('\n', ''))
    return list

stopwords = read_txt('english_stopwords.txt')
stemmer = SnowballStemmer('english')
def clean_accents(tweet):
    tweet = re.sub(r"[àáâãäå]", "a", tweet)
    tweet = re.sub(r"ç", "c", tweet)
    tweet = re.sub(r"[èéêë]", "e", tweet)
    tweet = re.sub(r"[ìíîï]", "i", tweet)
    tweet = re.sub(r"[òóôõö]", "o", tweet)
    tweet = re.sub(r"[ùúûü]", "u", tweet)
    tweet = re.sub(r"[ýÿ]", "y", tweet)

    return tweet

def clean_tweet(tweet, stem = False):
    tweet = tweet.lower().strip()
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    tweet = re.sub(r'http?:\/\/\S+', '', tweet)
    tweet = re.sub(r'www?:\/\/\S+', '', tweet)
    tweet = re.sub(r'\s([@#][\w_-]+)', "", tweet)
    tweet = re.sub(r"\n", " ", tweet)
    tweet = clean_accents(tweet)
    tweet = re.sub(r"\b(a*ha+h[ha]*|o?l+o+l+[ol]*|x+d+[x*d*]*|a*ja+[j+a+]+)\b", "<risas>", tweet)
    for symbol in punctuations:
        tweet = tweet.replace(symbol, "")
    tokens = []
    for token in tweet.strip().split():
        if token not in punctuations and token not in stopwords:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [3]:
# 	id	text_orig	ED_Patient	ProED	informative	scientific	hashtags	
# entities_instances_wikidata	spacy_entities_ids	spacy_entities_labels	
# falcon_spacy_entities	falcon_spacy_labels	falcon_spacy_embeddingsmd2_mw50_RW	
# falcon_spacy_embeddingsmd2_mw100_RW	sent_embedding_1	sent_embedding_2

cols = ['id','text_orig','ED_Patient','ProED','informative','scientific','hashtag','entities_instances_wikidata','spacy_entities_ids','spacy_entities_labels','falcon_spacy_entities'
       ,'falcon_spacy_labels','falcon_spacy_embeddingsmd2_mw50_RW','falcon_spacy_embeddingsmd2_mw100_RW','sent_embedding_1','sent_embedding_2']

In [4]:
tweets1 = tweets.copy()
# .drop(['entities_instances_wikidata','spacy_entities_ids','spacy_entities_labels'], axis=1)

tweets1['text_cleaned'] = tweets['text_orig'].apply(lambda s : clean_tweet(s))
#print(tweets1['text_cleaned'].head(5))

In [5]:
#tweets1.head(2)

In [6]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
df = tweets1.copy()
X = df['text_cleaned']
X_b = np.array(df['sent_embedding_1'].tolist())
X_b = np.asarray(X_b, dtype=np.float32)


#X = np.array(df['falcon_spacy_labels'].tolist())

Y1 = df['ED_Patient']
Y2 = df['ProED']
Y3 = df['informative']
Y4 = df['scientific']

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MICROSOFT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

Xc = np.zeros((1968,1222))
for i in range(1968):
    Xc[i] = np.concatenate((X[i],X_b[i]))
   

In [9]:
from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(Xc, Y1, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))


from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))

[[253  42]
 [ 41 255]]
              precision    recall  f1-score   support

           0       0.86      0.86      0.86       295
           1       0.86      0.86      0.86       296

    accuracy                           0.86       591
   macro avg       0.86      0.86      0.86       591
weighted avg       0.86      0.86      0.86       591

0.8595600676818951
[[268  27]
 [ 92 204]]
              precision    recall  f1-score   support

           0       0.74      0.91      0.82       295
           1       0.88      0.69      0.77       296

    accuracy                           0.80       591
   macro avg       0.81      0.80      0.80       591
weighted avg       0.81      0.80      0.80       591

0.7986463620981388


In [11]:
from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(Xc, Y2, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))


from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y2, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))

[[455  10]
 [ 62  64]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       465
           1       0.86      0.51      0.64       126

    accuracy                           0.88       591
   macro avg       0.87      0.74      0.78       591
weighted avg       0.88      0.88      0.87       591

0.8781725888324873
[[421  44]
 [ 39  87]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       465
           1       0.66      0.69      0.68       126

    accuracy                           0.86       591
   macro avg       0.79      0.80      0.79       591
weighted avg       0.86      0.86      0.86       591

0.8595600676818951


In [12]:
from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(Xc, Y3, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))


from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y3, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))

[[322  34]
 [ 59 176]]
              precision    recall  f1-score   support

           0       0.85      0.90      0.87       356
           1       0.84      0.75      0.79       235

    accuracy                           0.84       591
   macro avg       0.84      0.83      0.83       591
weighted avg       0.84      0.84      0.84       591

0.8426395939086294
[[332  24]
 [101 134]]
              precision    recall  f1-score   support

           0       0.77      0.93      0.84       356
           1       0.85      0.57      0.68       235

    accuracy                           0.79       591
   macro avg       0.81      0.75      0.76       591
weighted avg       0.80      0.79      0.78       591

0.7884940778341794


In [13]:
from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(Xc, Y4, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))


from sklearn.ensemble import RandomForestClassifier

X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y4, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X1_train, y1_train) 

y1_pred = classifier.predict(X1_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y1_test,y1_pred))
print(classification_report(y1_test,y1_pred))
print(accuracy_score(y1_test, y1_pred))

[[419  18]
 [ 26 128]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       437
           1       0.88      0.83      0.85       154

    accuracy                           0.93       591
   macro avg       0.91      0.89      0.90       591
weighted avg       0.92      0.93      0.92       591

0.9255499153976311
[[433   4]
 [ 70  84]]
              precision    recall  f1-score   support

           0       0.86      0.99      0.92       437
           1       0.95      0.55      0.69       154

    accuracy                           0.87       591
   macro avg       0.91      0.77      0.81       591
weighted avg       0.89      0.87      0.86       591

0.8747884940778342


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [15]:
df = tweets1.copy()

X_orig = df['text_cleaned']
X_b = np.array(df['sent_embedding_1'].tolist())
X_b = np.asarray(X_b, dtype=np.float32)

Y1 = df['ED_Patient']
Y2 = df['ProED']
Y3 = df['informative']
Y4 = df['scientific']

In [16]:
max_words = 1000
max_len = 100
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_orig)
sequences = tok.texts_to_sequences(X_orig)
X_c = sequence.pad_sequences(sequences,maxlen=max_len)

In [17]:
X_c.shape

(1968, 100)

In [18]:
X_b.shape

(1968, 100)

In [19]:
X = X_b + X_c

In [20]:
max_len=1000
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(100)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.1)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [21]:
from sklearn.metrics import confusion_matrix
'''y1_pred = model.predict(X1_test)
y1_pred = np.argmax(y1_pred, axis=1)
conf_mat = confusion_matrix(y1_test, y1_pred)'''
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [24]:
# CATEGORY 1
X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y1, test_size=0.3, random_state=42)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X1_train)
sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
test_sequences = tok.texts_to_sequences(X1_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
#accr = model.evaluate(test_sequences_matrix,y1_test)
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)

#print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))


X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.3, random_state=42)

max_words = 1000
max_len = 200
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X1_train)
#sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = X1_train
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#test_sequences = X1_test
test_sequences_matrix = X1_test
#accr = model.evaluate(test_sequences_matrix,y1_test)
loss, accuracy, f1_score, precision, recall = model.evaluate(X1_test, y1_test, verbose=0)

#print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))



Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 200)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 200, 100)          100000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               25856     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                

In [26]:
# CATEGORY 2
X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y2, test_size=0.3, random_state=42)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X1_train)
sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
test_sequences = tok.texts_to_sequences(X1_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))


X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y2, test_size=0.3, random_state=42)

max_words = 1000
max_len = 200
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X1_train)
#sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = X1_train
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#test_sequences = X1_test
test_sequences_matrix = X1_test
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)

#print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))


Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 200)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 200, 100)          100000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               80400     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               25856     
_________________________________________________________________
activation_16 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)               

In [29]:
# CATEGORY 3
X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y3, test_size=0.3, random_state=42)

max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X1_train)
sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
test_sequences = tok.texts_to_sequences(X1_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))



X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y3, test_size=0.3, random_state=42)

max_words = 1000
max_len = 200
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X1_train)
#sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = X1_train
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#test_sequences = X1_test
test_sequences_matrix = X1_test
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))




Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 200)]             0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 200, 100)          100000    
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               80400     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               25856     
_________________________________________________________________
activation_24 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)               

In [30]:
# CATEGORY 4

X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y4, test_size=0.3, random_state=42)


tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X1_train)
sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=8,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
test_sequences = tok.texts_to_sequences(X1_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))



X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y4, test_size=0.3, random_state=42)

max_words = 1000
max_len = 20
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X1_train)
#sequences = tok.texts_to_sequences(X1_train)
sequences_matrix = X1_train
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])
model.fit(sequences_matrix,y1_train,batch_size=128,epochs=10,
          validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#test_sequences = X1_test
test_sequences_matrix = X1_test
loss, accuracy, f1_score, precision, recall = model.evaluate(test_sequences_matrix, y1_test, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n fº {:0.3f}'.format(loss,accuracy,f1_score))








Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 20)]              0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 20, 100)           100000    
_________________________________________________________________
lstm_14 (LSTM)               (None, 100)               80400     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               25856     
_________________________________________________________________
activation_28 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)               

In [225]:
X.shape

(1968, 100)

In [35]:
import tensorflow as tf

In [36]:
# CATEGORY 1
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.3, random_state=42)

VOCAB_SIZE=1000


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1968,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y1, test_size=0.3, random_state=42)

VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(np.asarray(X1_train))

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))





Epoch 1/14
44/44 [==============================] - 3s 75ms/step - loss: 0.6873 - accuracy: 0.5251 - val_loss: 0.6795 - val_accuracy: 0.4992
Epoch 2/14
44/44 [==============================] - 1s 25ms/step - loss: 0.6433 - accuracy: 0.5251 - val_loss: 0.5582 - val_accuracy: 0.4992
Epoch 3/14
44/44 [==============================] - 1s 26ms/step - loss: 0.4750 - accuracy: 0.6935 - val_loss: 0.4228 - val_accuracy: 0.7868
Epoch 4/14
44/44 [==============================] - 1s 26ms/step - loss: 0.3929 - accuracy: 0.7996 - val_loss: 0.4447 - val_accuracy: 0.7902
Epoch 5/14
44/44 [==============================] - 1s 26ms/step - loss: 0.3536 - accuracy: 0.8199 - val_loss: 0.3844 - val_accuracy: 0.8342
Epoch 6/14
44/44 [==============================] - 1s 25ms/step - loss: 0.3109 - accuracy: 0.8620 - val_loss: 0.3793 - val_accuracy: 0.8342
Epoch 7/14
44/44 [==============================] - 1s 26ms/step - loss: 0.2874 - accuracy: 0.8794 - val_loss: 0.4218 - val_accuracy: 0.8257
Epoch 8/14
44

In [37]:
# CATEGORY 2
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y2, test_size=0.3, random_state=42)

VOCAB_SIZE=1000


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1968,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y2, test_size=0.3, random_state=42)

VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(np.asarray(X1_train))

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))




Epoch 1/14
44/44 [==============================] - 3s 57ms/step - loss: 0.6695 - accuracy: 0.7618 - val_loss: 0.6091 - val_accuracy: 0.7868
Epoch 2/14
44/44 [==============================] - 1s 14ms/step - loss: 0.5122 - accuracy: 0.7618 - val_loss: 0.4479 - val_accuracy: 0.7868
Epoch 3/14
44/44 [==============================] - 1s 14ms/step - loss: 0.4358 - accuracy: 0.7618 - val_loss: 0.3943 - val_accuracy: 0.7868
Epoch 4/14
44/44 [==============================] - 1s 14ms/step - loss: 0.3525 - accuracy: 0.7618 - val_loss: 0.3299 - val_accuracy: 0.7868
Epoch 5/14
44/44 [==============================] - 1s 14ms/step - loss: 0.2789 - accuracy: 0.7959 - val_loss: 0.3037 - val_accuracy: 0.8240
Epoch 6/14
44/44 [==============================] - 1s 14ms/step - loss: 0.2421 - accuracy: 0.8867 - val_loss: 0.3013 - val_accuracy: 0.8426
Epoch 7/14
44/44 [==============================] - 1s 14ms/step - loss: 0.2028 - accuracy: 0.9063 - val_loss: 0.3078 - val_accuracy: 0.8545
Epoch 8/14
44

In [38]:
# CATEGORY 3

X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y3, test_size=0.3, random_state=42)

VOCAB_SIZE=1000


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1968,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y3, test_size=0.3, random_state=42)

VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(np.asarray(X1_train))

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))




Epoch 1/14
44/44 [==============================] - 3s 74ms/step - loss: 0.6830 - accuracy: 0.6209 - val_loss: 0.6729 - val_accuracy: 0.6024
Epoch 2/14
44/44 [==============================] - 1s 19ms/step - loss: 0.6511 - accuracy: 0.6209 - val_loss: 0.6379 - val_accuracy: 0.6024
Epoch 3/14
44/44 [==============================] - 1s 20ms/step - loss: 0.5607 - accuracy: 0.6209 - val_loss: 0.5128 - val_accuracy: 0.6024
Epoch 4/14
44/44 [==============================] - 1s 20ms/step - loss: 0.4488 - accuracy: 0.6899 - val_loss: 0.4718 - val_accuracy: 0.8088
Epoch 5/14
44/44 [==============================] - 1s 20ms/step - loss: 0.3482 - accuracy: 0.8649 - val_loss: 0.4809 - val_accuracy: 0.8291
Epoch 6/14
44/44 [==============================] - 1s 20ms/step - loss: 0.2847 - accuracy: 0.8845 - val_loss: 0.4603 - val_accuracy: 0.8460
Epoch 7/14
44/44 [==============================] - 1s 20ms/step - loss: 0.2387 - accuracy: 0.9070 - val_loss: 0.4646 - val_accuracy: 0.8494
Epoch 8/14
44

In [40]:
# CATEGORY 4
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y4, test_size=0.3, random_state=42)

VOCAB_SIZE=1000


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1968,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


X1_train, X1_test, y1_train, y1_test = train_test_split(X_orig, Y4, test_size=0.3, random_state=42)

VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(np.asarray(X1_train))

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(2e-4),
              metrics=['accuracy'])

history = model.fit(X1_train,y1_train, epochs=14,
                    validation_data=(X1_test,y1_test), 
                    validation_steps=30)

test_loss, test_acc = model.evaluate(X1_test,y1_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))







Epoch 1/14
44/44 [==============================] - 3s 73ms/step - loss: 0.6698 - accuracy: 0.7603 - val_loss: 0.6253 - val_accuracy: 0.7394
Epoch 2/14
44/44 [==============================] - 1s 33ms/step - loss: 0.5572 - accuracy: 0.7603 - val_loss: 0.5521 - val_accuracy: 0.7394
Epoch 3/14
44/44 [==============================] - 1s 33ms/step - loss: 0.4967 - accuracy: 0.7603 - val_loss: 0.4981 - val_accuracy: 0.7394
Epoch 4/14
44/44 [==============================] - 1s 33ms/step - loss: 0.4141 - accuracy: 0.7603 - val_loss: 0.3894 - val_accuracy: 0.7394
Epoch 5/14
44/44 [==============================] - 1s 33ms/step - loss: 0.3075 - accuracy: 0.8562 - val_loss: 0.3088 - val_accuracy: 0.8849
Epoch 6/14
44/44 [==============================] - 1s 33ms/step - loss: 0.2257 - accuracy: 0.9303 - val_loss: 0.2926 - val_accuracy: 0.8917
Epoch 7/14
44/44 [==============================] - 1s 33ms/step - loss: 0.1817 - accuracy: 0.9477 - val_loss: 0.2905 - val_accuracy: 0.8934
Epoch 8/14
44